In [123]:
from __future__ import division
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.utils import class_weight
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn import neighbors
from sklearn import neural_network
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree
from sklearn.feature_extraction import DictVectorizer

In [124]:
def fit_and_predict(modelName, model, train_x, train_y):
    k = 10
    scores = cross_val_score(model, train_x, train_y, cv = k)
    hit_hate = np.mean(scores)
    
    msg = "Hit Rate from {0}: {1}".format(modelName, hit_hate)
    print(msg)
    
    return hit_hate

def gettingDistributionOfDatas():
    winners = int(list(Y_labels).count(1))
    print('Quantity winners: ' +str(winners))
    
    losers = int(list(Y_labels).count(0))
    print('Quantity losers: ' +str(losers))
    
    total = len(Y_labels)
    print('Total Quantity: ' +str(total))
    
    distribution_winners = int(winners/total * 100)
    distribution_losers = int(losers/total * 100)
    print('\nDistribution of winning data: ' +str(distribution_winners) + '%')
    print('Distribution of losing data: ' +str(distribution_losers) + '% \n')
    
    print('class_weight:')
    class_weight_count = {1: distribution_winners, 0: distribution_losers}
    print(class_weight_count)
    
    return class_weight_count

In [125]:
#get dataset to train/test and to predict
df = pd.read_csv('datasets/the_emmy_awards.csv')
#df_to_predict = pd.read_csv('datasets/Dados_Actress.csv')

In [126]:
df.head()

,year,category,nominee,staff,company,producer,win,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,2020,Outstanding Animated Program,Big Mouth,"Nick Kroll, Executive Producer; Andrew Goldber...",Netflix,A Netflix Original Production,NaN,NaN,NaN,NaN
1,2020,Outstanding Animated Program,Bob’s Burgers,"Loren Bouchard, Jim Dauterive, Loren Bouchard",FOX,"Jim Dauterive, Loren Bouchard",NaN,NaN,NaN,NaN
2,2020,Outstanding Animated Program,BoJack Horseman,"Raphael Bob-Waksberg, Executive Producer/Writt...",Netflix,"Tornante Productions, LLC",NaN,NaN,NaN,NaN
3,2020,Outstanding Animated Program,Rick And Morty,"Dan Harmon, Executive Producer; Justin Roiland...",Adult Swim,Williams Street Productions,NaN,NaN,NaN,NaN
4,2020,Outstanding Animated Program,The Simpsons,"Hank Azaria, as Moe, Carl, Duffman, Kirk",FOX,Gracie Films in association with 20th Century ...,NaN,NaN,NaN,NaN


In [127]:
category = 'Outstanding Limited Series'
df.loc[df['category'] == category]

,year,category,nominee,staff,company,producer,win,Unnamed: 7,Unnamed: 8,Unnamed: 9
5,2020,Outstanding Limited Series,Little Fires Everywhere,Celeste Ng,Hulu,"Best Day Ever Productions, Simpson Street, Hel...",NaN,NaN,NaN,NaN
6,2020,Outstanding Limited Series,Mrs. America,"Tanya Barfield, Boo Killebrew, Sharon Hoffman",FX Networks,"Tanya Barfield, Boo Killebrew, Sharon Hoffman",NaN,NaN,NaN,NaN
7,2020,Outstanding Limited Series,Unbelievable,"John Vohlers, T. Christian Miller, Ken Armstro...",Netflix,"John Vohlers, T. Christian Miller, Ken Armstro...",NaN,NaN,NaN,NaN
8,2020,Outstanding Limited Series,Unorthodox,Maria Schrader,Netflix,Maria Schrader,NaN,NaN,NaN,NaN
9,2020,Outstanding Limited Series,Watchmen,"White Rabbit, Paramount Television, DC Enterta...",HBO,"White Rabbit, Paramount Television, DC Enterta...",NaN,NaN,NaN,NaN
507,2019,Outstanding Limited Series,Escape At Dannemora,"Ben Stiller, Executive Producer; Michael Tolki...",Showtime,"Red Hour, Busyhands, The White Mountain Compan...",False,NaN,NaN,NaN
508,2019,Outstanding Limited Series,Fosse/Verdon,"Steven Levenson, Executive Producer; Thomas Ka...",FX Networks,Fox 21 Television Studios and FX Productions,False,NaN,NaN,NaN
509,2019,Outstanding Limited Series,When They See Us,"Ava DuVernay, Executive Producer; Jeff Skoll, ...",Netflix,"Participant Media, Tribeca Productions, Harpo ...",False,NaN,NaN,NaN
537,2019,Outstanding Limited Series,Sharp Objects,"Marti Noxon, Executive Producer; Jason Blum, E...",HBO,HBO Entertainment in association with eOne and...,False,NaN,NaN,NaN
703,2019,Outstanding Limited Series,Chernobyl,"Craig Mazin, Executive Producer; Carolyn Strau...",HBO,HBO Miniseries and SKY in association with Sis...,True,NaN,NaN,NaN


In [128]:
df.columns

Index(['year', 'category', 'nominee', 'staff', 'company', 'producer', 'win',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'],
      dtype='object')

In [129]:
del df['Unnamed: 7']
del df['Unnamed: 8']
del df['Unnamed: 9']

In [130]:
def separateProducers(producer):
    producer = str(producer).replace(" and", ",")
    producer = producer.replace(" in association with", ",")    
    return producer

df['producer'] = df['producer'].apply(separateProducers)

In [134]:
def getMaxProducers(producer):
    totalProducers = len(str(producer).split(','))
    return totalProducers
    
df['totalProducers'] = df['producer'].apply(getMaxProducers)

In [138]:
df['totalStaffs'] = df['staff'].apply(getMaxProducers)

In [139]:
df['totalProducers'].max()

9

In [140]:
df['totalStaffs'].max()

38

In [93]:
df['totalProducers'].mean()

1.512179278986684

In [94]:
producersList = [[],[],[],[],[],[],[],[],[]]

for row in df.iterrows():
    producers = row[1]['producer'].split(',')
    for i in range(0, 9):
        if(i+1 <= len(producers)):
            producersList[i].append(producers[i])
        else:
            producersList[i].append(producers[0])
            

In [95]:
df['producer1'] = producersList[0]
df['producer2'] = producersList[1]
df['producer3'] = producersList[2]
df['producer4'] = producersList[3]
df['producer5'] = producersList[4]
df['producer6'] = producersList[5]
df['producer7'] = producersList[6]
df['producer8'] = producersList[7]
df['producer9'] = producersList[8]

In [96]:
def getMaxNominees(name):

    dictNominees = {}
    maxNominees = []

    for row in df.iterrows():
        producers = set(str(row[1][name]).split(','))
        for i in producers:
            if i in dictNominees:
                dictNominees[i] = dictNominees[i] + 1
            else:
                dictNominees[i] = 1

    for row in df.iterrows():
        producers = set(str(row[1][name]).split(','))
        maxNominee = 0
        for i in producers:
            if(dictNominees[i] > maxNominee):
                maxNominee = dictNominees[i]
        maxNominees.append(maxNominee)

    nameMaxNominees = 'maxNominees_' + name 
    df[nameMaxNominees] = maxNominees

In [97]:
getMaxNominees('producer')

In [98]:
getMaxNominees('nominee')

In [99]:
getMaxNominees('company')

In [100]:
category = 'Outstanding Comedy Series'

df = df.loc[df['category'] == category]

listDummies = ['nominee', 'company', 'producer1', 'producer2', 'producer3', 
      'producer4', 'producer5', 'producer6', 'producer7', 'producer8', 'producer9']

listColumns = ['nominee', 'company', 'totalProducers', 'producer1', 'producer2', 'producer3', 
      'producer4', 'producer5', 'producer6', 'producer7', 'producer8', 'producer9', 
      'maxNominees_producer', 'maxNominees_nominee', 'maxNominees_company', 'year', 'win']

listColumnsX = ['nominee', 'company', 'totalProducers', 'producer1', 'producer2', 'producer3', 
      'producer4', 'producer5', 'producer6', 'producer7', 'producer8', 'producer9', 
      'maxNominees_producer', 'maxNominees_nominee', 'maxNominees_company']

X = df[listColumns]

X_previous_years = X.loc[X['year'] < 2020]
X_actual_year = X.loc[X['year'] == 2020]

#getting the importants attributes from original dataset to our dataset to train and test 
X_train_test = X_previous_years[listColumnsX]
Y_labels = X_previous_years['win']

#getting the importants attributes from original dataset to our dataset to predict
X_to_predict = X_actual_year[listColumnsX]
Y_to_predict = X_actual_year[listColumnsX]

# to improve our analysis, we will do a pre-processing to normalize the data
attributes_to_normalize = ['totalProducers', 'maxNominees_producer', 'maxNominees_nominee', 'maxNominees_company']
X_train_robust = X_train_test.copy()
X_train_to_predict = X_to_predict

minMax = MinMaxScaler().fit(X_train_test[attributes_to_normalize])

X_train_robust[attributes_to_normalize] = minMax.transform(X_train_test[attributes_to_normalize])

In [101]:
X_train_to_predict[["nominee", "company", "totalProducers", "producer1", "maxNominees_producer", "maxNominees_nominee", "maxNominees_company"]]

,nominee,company,totalProducers,producer1,maxNominees_producer,maxNominees_nominee,maxNominees_company
42,Curb Your Enthusiasm,HBO,1,HBO Entertainment,497,44,2577
43,Dead to Me,Netflix,1,CBS Television Studios,63,3,479
44,The Good Place,Netflix,3,Universal Television,131,9,479
45,Insecure,HBO,3,Issa Rae Productions,120,5,2577
46,The Kominsky Method,Netflix,2,Chuck Lorre Productions,433,5,479
47,The Marvelous Mrs. Maisel,Prime Video,3,Dorothy Parker Drank Here Productions,29,35,70
48,Schitt's Creek,CBC,1,Not a Real Company Productions,3,7,1
49,What We Do in the Shadows,FX Networks,2,Two Canoes Pictures,2,1,401


In [102]:
Y_labels = Y_labels.values
Y_to_predict = Y_to_predict.values

In [103]:
def convertYLabels(value):
    if value == 'True':
        return 1
    else:
        return 0

In [104]:
for i in range(0, len(Y_labels)):
    Y_labels[i] = convertYLabels(Y_labels[i])
    
for i in range(0, len(Y_to_predict)):
    Y_to_predict[i] = convertYLabels(str(Y_to_predict[i]))

In [105]:
gettingDistributionOfDatas()

Quantity winners: 48
Quantity losers: 208
Total Quantity: 256

Distribution of winning data: 18%
Distribution of losing data: 81% 

class_weight:
{1: 18, 0: 81}


{1: 18, 0: 81}

In [108]:
# categorical values treatment
X_train_dict = X_train_robust.to_dict(orient='records')  # turn each row as key-value
X_to_predict_dict = X_train_to_predict.to_dict(orient='records')

# DictVectorizer
dv_X = DictVectorizer(sparse=False).fit(X_train_dict)
# sparse = False makes the output is not a sparse matrix apply dv_X on X_dict
X_train = dv_X.transform(X_train_dict)
X_to_predict = dv_X.transform(X_to_predict_dict)

In [109]:
X_train_test = X_train
X_to_predict = X_to_predict
Y_labels = Y_labels

In [110]:
#Percentage train
percentage_train = 0.8
size_train = percentage_train * len(Y_labels)
train_data_X = X_train_test[:int(size_train)]
train_data_Y = Y_labels[:int(size_train)]

In [111]:
#Percentage test
test_data_X = X_train_test[int(size_train):]
test_data_Y = Y_labels[int(size_train):]

In [112]:
#Counting quantity of predictions as won
qt_candidates = len(X_to_predict)
candidate = [0] * qt_candidates
candidate0 = [0] * qt_candidates
results = {}

In [117]:
def predict_results(model, result):
    results[result] = model
    model.fit(train_data_X, train_data_Y)
    resultEmmy = model.predict(X_to_predict)
    print('Emmy 2020: ' + str(resultEmmy))
    print('')
    for i in range(qt_candidates):
            if(resultEmmy[i] == 1.0):
                candidate0[i] = candidate0[i] + 1
    if(result > 0.79):
        for i in range(qt_candidates):
            if(resultEmmy[i] == 1.0):
                candidate[i] = candidate[i] + 1

In [114]:
train_data_Y = train_data_Y.astype(int)
test_data_Y = test_data_Y.astype(int)

In [115]:
#Predict Adaboost Grided
model = AdaBoostClassifier(n_estimators= 45, learning_rate= 0.01)
result = fit_and_predict("AdaBoostClassifier Grided", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from AdaBoostClassifier Grided: 0.8192982456140351
Oscar 2019: [0 0 0 0 0 0 0 0]



In [116]:
#Predict Adaboost
model = AdaBoostClassifier()
result = fit_and_predict("AdaBoostClassifier", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from AdaBoostClassifier: 0.7491228070175439
Oscar 2019: [0 0 0 0 0 1 0 0]



In [22]:
# Predict SVC Grided
model = svm.SVC(C= 0.03, max_iter= -1, decision_function_shape= 'ovo', tol= 0.001, class_weight= None, gamma='scale')
result = fit_and_predict("SVC Grided", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from SVC Grided: 0.7921428571428571
Oscar 2019: [0. 0. 0. 0. 0.]



In [23]:
# Predict SVC Grided with class_weight balanced
model = svm.SVC(C= 0.03, max_iter= -1, decision_function_shape= 'ovo', tol= 0.001, class_weight= 'balanced', gamma='auto')
result = fit_and_predict("SVC Grided with class_weight balanced", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from SVC Grided with class_weight balanced: 0.37214285714285716
Oscar 2019: [1. 1. 1. 1. 1.]



In [24]:
# Predict SVC Grided with class_weight count
model = svm.SVC(C= 0.03, max_iter= -1, decision_function_shape= 'ovo', tol= 0.001, class_weight= class_weight_dict, gamma='auto')
result = fit_and_predict("SVC Grided with class_weight count", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from SVC Grided with class_weight count: 0.7921428571428571
Oscar 2019: [0. 0. 0. 0. 0.]



In [25]:
# Predict SVC
model = svm.SVC(gamma='auto')
result = fit_and_predict("SVC Grided", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from SVC Grided: 0.8197619047619048
Oscar 2019: [0. 0. 0. 0. 0.]



In [26]:
#Predict KNN Grided
model = neighbors.KNeighborsClassifier(n_neighbors = 4, metric = 'euclidean', weights = 'uniform', algorithm = 'auto')
result = fit_and_predict("KNN Grided", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from KNN Grided: 0.8260119047619048
Oscar 2019: [0. 0. 0. 0. 0.]



In [27]:
#Predict KNN
model = neighbors.KNeighborsClassifier()
result = fit_and_predict("KNN", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from KNN: 0.8322619047619046
Oscar 2019: [0. 0. 0. 0. 0.]



In [28]:
#Predict MLP
model = neural_network.MLPClassifier()
result = fit_and_predict("MLP", model, train_data_X, train_data_Y)
predict_results(model, result)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr

Hit Rate from MLP: 0.8192261904761905
Oscar 2019: [0. 0. 0. 0. 0.]



/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [29]:
#Predict Logistic Regression and class_weigh balanced
model = LogisticRegression(C= 0.01, fit_intercept= True, tol= 1e-05, max_iter= 100, class_weight= 'balanced', solver='lbfgs')
result = fit_and_predict("LogisticRegression Grided and class_weigh balanced", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from LogisticRegression Grided and class_weigh balanced: 0.7426190476190476
Oscar 2019: [1. 0. 1. 1. 0.]



In [30]:
#Predict Logistic Regression and class_weigh count
model = LogisticRegression(C= 0.01, fit_intercept= True, tol= 1e-05, max_iter= 100, class_weight= class_weight_dict, solver='lbfgs')
result = fit_and_predict("LogisticRegression Grided and class_weigh balanced", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from LogisticRegression Grided and class_weigh balanced: 0.7921428571428571
Oscar 2019: [0. 0. 0. 0. 0.]



In [31]:
#Predict Logistic Regression
model = LogisticRegression(solver='lbfgs')
result = fit_and_predict("LogisticRegression", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from LogisticRegression: 0.825952380952381
Oscar 2019: [0. 1. 0. 0. 0.]



In [32]:
#Predict Bagging Grided
model = BaggingClassifier(n_estimators= 42, max_samples= 0.087)
result = fit_and_predict("BaggingClassifier Grided", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from BaggingClassifier Grided: 0.8130952380952381
Oscar 2019: [0. 0. 0. 0. 0.]



In [33]:
#Predict Bagging
model = BaggingClassifier()
result = fit_and_predict("BaggingClassifier", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from BaggingClassifier: 0.826845238095238
Oscar 2019: [0. 0. 1. 0. 0.]



In [34]:
#Predict Gradient Boosting
model = GradientBoostingClassifier()
result = fit_and_predict("GradientBoostingClassifier", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from GradientBoostingClassifier: 0.7997023809523809
Oscar 2019: [0. 1. 1. 0. 0.]



In [35]:
#Predict Random Forest Grided
model = RandomForestClassifier(n_estimators= 10, criterion= 'gini', max_depth= 3)
result = fit_and_predict("RandomForestClassifier Grided", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from RandomForestClassifier Grided: 0.8202380952380952
Oscar 2019: [0. 0. 0. 0. 0.]



In [36]:
#Predict Random Forest Grided and class_weight count
model = RandomForestClassifier(n_estimators= 10000, criterion= 'gini', max_depth= 3, class_weight= class_weight_dict)
result = fit_and_predict("RandomForestClassifier Grided and class_weight count", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from RandomForestClassifier Grided and class_weight count: 0.7921428571428571
Oscar 2019: [0. 0. 0. 0. 0.]



In [37]:
#Predict Random Forest Grided and class_weight balanced
model = RandomForestClassifier(n_estimators= 10000, criterion= 'gini', max_depth= 3, class_weight= 'balanced')
result = fit_and_predict("RandomForestClassifier Grided and class_weight balanced", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from RandomForestClassifier Grided and class_weight balanced: 0.8126190476190477
Oscar 2019: [0. 0. 1. 0. 0.]



In [38]:
#Predict Random Forest Grided
model = RandomForestClassifier(n_estimators=100)
result = fit_and_predict("RandomForestClassifier", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from RandomForestClassifier: 0.8335714285714285
Oscar 2019: [0. 0. 0. 0. 0.]



In [39]:
#Predict Extra Tress Grided
model = ExtraTreesClassifier(criterion = 'gini', max_depth = 3, n_estimators = 150)
result = fit_and_predict("ExtraTreesClassifier Grided", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from ExtraTreesClassifier Grided: 0.8202380952380952
Oscar 2019: [0. 0. 0. 0. 0.]



In [40]:
#Predict Extra Tress Grided and class_weight balanced
model = ExtraTreesClassifier(criterion = 'gini', max_depth = 3, n_estimators = 150, class_weight= 'balanced')
result = fit_and_predict("ExtraTreesClassifier Grided and class_weight balanced", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from ExtraTreesClassifier Grided and class_weight balanced: 0.8392857142857142
Oscar 2019: [0. 1. 1. 0. 0.]



In [41]:
#Predict Extra Tress
model = ExtraTreesClassifier(n_estimators = 100)
result = fit_and_predict("ExtraTreesClassifier", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from ExtraTreesClassifier: 0.8464285714285713
Oscar 2019: [0. 1. 1. 0. 0.]



In [42]:
#Predict DecisionTree Grided
model = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 3)
result = fit_and_predict("DecisionTree Grided", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from DecisionTree Grided: 0.79125
Oscar 2019: [0. 0. 0. 0. 0.]



In [43]:
#Predict DecisionTree Grided and class_weight count
model = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 3, class_weight= class_weight_dict)
result = fit_and_predict("DecisionTree Grided and class_weight count", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from DecisionTree Grided and class_weight count: 0.7516666666666667
Oscar 2019: [0. 0. 0. 0. 0.]



In [44]:
#Predict DecisionTree Grided and class_weight balanced
model = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 3, class_weight = 'balanced')
result = fit_and_predict("DecisionTree Grided and class_weight balanced", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from DecisionTree Grided and class_weight balanced: 0.7045833333333333
Oscar 2019: [0. 1. 1. 0. 0.]



In [45]:
#Predict DecisionTree
model = tree.DecisionTreeClassifier()
result = fit_and_predict("DecisionTree", model, train_data_X, train_data_Y)
predict_results(model, result)

Hit Rate from DecisionTree: 0.7382142857142858
Oscar 2019: [0. 0. 1. 0. 0.]



In [46]:
#The effectiveness of the algorithm that kicks everything 0 or 1 or a single value
base_hit = max(Counter(test_data_Y).values()) #Devolve a quantidade do maior elemento
base_one = list(Y_labels).count(1)
base_zero = list(Y_labels).count(0)
hit_rate_base = 100.0 * base_hit / len(test_data_Y)
print("Hit rate based on validation data: %f" %hit_rate_base)

Hit rate based on validation data: 81.578947


In [47]:
maximum = max(results)
winner = results[maximum]
print('\n\n')
print(winner)
print('\n\n')
winner.fit(train_data_X, train_data_Y)
result = winner.predict(test_data_X)

len_to_predict = len(train_data_Y)
hit_rate = metrics.accuracy_score(test_data_Y, result)

print("Better algorithm hit rate in the real world " + "was: " 
      + str(hit_rate) + "% " + "from " + str(len_to_predict) + " elements\n\n")




ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)



Better algorithm hit rate in the real world was: 0.9473684210526315% from 149 elements




In [48]:
name_and_films = df_to_predict[['film', 'name']]
print(name_and_films)
print('\n')
winner.fit(train_data_X, train_data_Y)
winner_result = winner.predict(X_to_predict)
print('\nBest model predict:')
print(winner_result)
print(name_and_films.iloc[[winner_result.tolist().index(max(winner_result))]])

                        film              name
0                       Roma  Yalitza Aparicio
1                   The Wife       Glenn Close
2              The Favourite     Olivia Colman
3             A Star Is Born         Lady Gaga
4  Can You Ever Forgive Me?   Melissa McCarthy



Best model predict:
[0. 1. 1. 0. 0.]
       film         name
1  The Wife  Glenn Close


In [49]:
print('\nWithout accuracy validation:')
print(candidate0)
print(name_and_films.iloc[[candidate0.index(max(candidate0))]])
print("\n")
print('\nOnly if accuracy > 89%:')
print(candidate)
print(name_and_films.iloc[[candidate.index(max(candidate))]])


Without accuracy validation:
[2, 7, 10, 2, 1]
            film           name
2  The Favourite  Olivia Colman



Only if accuracy > 89%:
[0, 5, 6, 0, 0]
            film           name
2  The Favourite  Olivia Colman
